Motivation:
Traditional RAG systems often use dense vector embeddings for retrieval, which can be computationally expensive and may not always capture the nuances of term importance. BM25 RAG addresses these limitations by using a probabilistic retrieval model that considers term frequency, inverse document frequency, and document length. This approach can lead to more accurate and interpretable retrieval, especially for queries requiring specific or rare information.

https://github.com/adithya-s-k/AI-Engineering.academy/blob/main/RAG/01_BM25_RAG/notebook.ipynb

In [5]:
%pip install qdrant_client

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 267 kB 511 kB/s eta 0:00:01
     |████████████████████████████████| 5.6 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 11.1 MB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 73 kB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 431 kB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 417 kB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 691 kB/s eta 0:00:01
     |████████████████████████████████| 93 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.8 MB 802 kB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use 

In [6]:
import logging
import sys
import os

from IPython.display import Markdown, display
import qdrant_client

/Users/sseadmin/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [8]:
%pip install llama_index
%pip install llama-index-embeddings-fastembed

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.5 MB 627 kB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 454 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 455 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 983 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 52 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 92 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 242 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 11.3 MB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 2

In [9]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings

In [11]:
%pip install llama-index-embeddings-fastembed

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install llama-index-vector-stores-qdrant

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [13]:
from llama_index.vector_stores.qdrant import QdrantVectorStore

In [14]:
from llama_index.embeddings.fastembed import FastEmbedEmbedding

/Users/sseadmin/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
%pip install llama_index.llms.ollama

In [18]:
%pip install llama_index.embeddings.ollama

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [19]:
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

In [21]:
Settings.llm = Ollama(
    model="llama3.2", 
    temperature=0.1,
    context_window=8096,  # equivalent to max_tokens
    streaming=True
)

In [22]:
Settings.embed_model = OllamaEmbedding(
    model_name="nomic-embed-text",
    embed_batch_size=10
)

In [23]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader("./data", recursive=True).load_data(show_progress=True)

Loading files: 100%|██████████| 1/1 [00:00<00:00,  2.34file/s]


In [25]:
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.core.ingestion import IngestionPipeline

pipeline = IngestionPipeline(
    transformations=[
        MarkdownNodeParser(include_metadata=True),
        # TokenTextSplitter(chunk_size=500, chunk_overlap=20),
        # SentenceSplitter(chunk_size=1024, chunk_overlap=20),
        # SemanticSplitterNodeParser(buffer_size=1, breakpoint_percentile_threshold=95 , embed_model=Settings.embed_model),
        Settings.embed_model,
    ],
)

# Ingest directly into a vector db
nodes = pipeline.run(documents=documents , show_progress=True)
print("Number of Nodes:",len(nodes))

Generating embeddings: 100%|██████████| 15/15 [01:36<00:00,  6.44s/it]

Number of Nodes: 15


In [26]:
import asyncio
from llama_index.core.storage.docstore import SimpleDocumentStore

docstore = SimpleDocumentStore()
docstore.add_documents(nodes)
docstore.persist(persist_path="./docstore.json")

In [31]:
%pip install llama-index-retrievers-bm25

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 52 kB 663 kB/s eta 0:00:01
     |████████████████████████████████| 220 kB 831 kB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [32]:
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer

In [33]:
# We can pass in the index, docstore, or list of nodes to create the retriever
bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=2,
    # Optional: We can pass in the stemmer and set the language for stopwords
    # This is important for removing stopwords and stemming the query + text
    # The default is english for both
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

In [35]:
%pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 7.8 MB 878 kB/s eta 0:00:01
     |████████████████████████████████| 249 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 106 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 675 kB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [36]:
from llama_index.core.response.notebook_utils import display_source_node


Generating embeddings:  60%|██████    | 9/15 [12:34<08:22, 83.79s/it]
Matplotlib is building the font cache; this may take a moment.


In [37]:
# will retrieve context from specific companies
retrieved_nodes = bm25_retriever.retrieve(
    "Why is Rainfall forecast imperative as overwhelming precipitation can lead to numerous catastrophes"
)
for node in retrieved_nodes:
    display_source_node(node, source_length=5000)

**Node ID:** cdef3399-50b8-4a0f-a25f-24d4d04eafdf<br>**Similarity:** 5.760929107666016<br>**Text:** Development of Multiple Combined Regression
Methods for Rainfall Measurement.
Nusrat Jahan Prottasha1, Md. Jashim Uddin 2, Md. Kowsher3, Rokeya Khatun
Shorna4, Niaz Al Murshed 5, and Boktiar Ahmed Bappy 6
1 Daﬀodil International University Dhaka 1207, Bangladesh,
jahannusratprotta@gmail.com
2 Noakhali Science and Technology University, 3814, Dhaka,
mdjaud12@gmail.com
3 Stevens Institute of Technology, Hoboken, NJ 07030 USA,
ga.kowsher@gmail.com
4 Daﬀodil International University, 1207, Dhaka,
rokeyashorna5@gmail.com
5 Jahangirnagar University, 1342, Dhaka,
niazalmurshed.ai@gmail.com
6 Jhenaidah polytechnic institute, 7300, Dhaka,
entbappy73@gmail.com
Abstract. Rainfall forecast is imperative as overwhelming precipitation
can lead to numerous catastrophes. The prediction makes a diﬀerence for
individuals to require preventive measures. In addition, the expectation
ought to be precise. Most of the nations in the world is an agricultural
nation and most of the economy of any nation depends upon agriculture.
Rain plays an imperative part in agribusiness so the early expectation of
rainfall plays a vital part within the economy of any agricultural. Over-
whelming precipitation may well be a major disadvantage. It’s a cause
for natural disasters like ﬂoods and drought that unit of measurement
experienced by people over the world each year. Rainfall forecast has
been one of the foremost challenging issues around the world in the ﬁnal
year. There are so many techniques that have been invented for predict-
ing rainfall but most of them are classiﬁcation, clustering techniques.
Predicting the quantity of rain prediction is crucial for countries’ people.
In our paperwork, we have proposed some regression analysis techniques
which can be utilized for predicting the quantity of rainfall (The amount
of rainfall recorded for the day in mm) based on some historical weather
conditions dataset. we have applied 10 supervised regressors (Machine
Learning Model) and some preprocessing methodology to the dataset.
We have also analyzed the result and compared them using various sta-
tistical parameters among these trained models to ﬁnd the bestperformed
model. Using this model for predicting the quantity of rainfall in some
diﬀerent places. Finally, the Random Forest regressor has predicted the
best r2 score of 0.869904217, and the mean absolute error is 0.194459262,
mean squared error is 0.126358647 and the root mean squared error is
0.355469615. . .<br>

**Node ID:** 9fd739ac-06c2-4fc3-a8d8-44ae31442bad<br>**Similarity:** 3.0598549842834473<br>**Text:** 2 Nusrat Jahan et al.
Keywords: Rainfall, Supervised Learning, Regression, Random Forest
Tree, AdaBoost Regressor, Gradient Boosting Regressor, XGBoost.
1 Introduction
This research paper proposed a scientiﬁc method to predict rainfall quantity
based on some diﬀerent weather conditions considering preceding weather records
and present weather situations using some regression analysis techniques .[1]
Rainfall determining is exceptionally vital since overwhelming and irregular rain-
fall can have numerous impacts on many other things like annihilation of river-
bank, crops, agriculture, and farms. One of the very deleterious departures is
ﬂooding due to the over rain.[2] According to Wikipedia in late summer 2002,
enormous storm downpours driven to gigantic ﬂooding in eastern India, Nepal,
and Bangladesh, killing over 500 individuals and clearing out millions of houses.
Each year in Bangladesh approximately 26,000 square kilometers (10,000 sq mi)
(around 18% of the country) is ﬂooded, killing over 5,000 individuals and wreck-
ing more than 7 million homes. On the other hand, Western Sydney is now
the ”greatest concern” from the worst ﬂoods in decades to have ravaged east-
ern Australia.[3] Jonh C, Rodda et al. presented a very rational method of the
rainfall measurement problem. The application of science and innovation that
predicts the state of the environment at any given speciﬁc period is known as
climate determining or weather forecasting. There are many distinctive strate-
gies for climate estimate and weather forecasting. But rainfall prediction is rare.
Some of the research has shown some classiﬁcation method to predict whether
it would be rain tomorrow or not. But instead of a classiﬁcation method for pre-
dicting rain, we need to the quantity of the rainfall in a particular place. There
is numerous equipment implement for foreseeing rainfall by utilizing the climate
conditions like temperature, humidity, weight. These conventional strategies can-
not work productively so by utilizing machine learning procedures. we can create
an exact comes about rain forecast. Ready to fair do it by having the histori-
cal information investigation of rainfall and can anticipate the precipitation for
future seasons. In our paper, we presented some predictive regression analysis
techniques to quantify rainfall quantity at a place. Here we used more than 10
years of historical data to train our model. The dataset contains various weather
conditions of diﬀerent places. This method can be utilized to predict the rainfall
(The amount of rainfall recorded for the day in mm) and avoid the annihilation
caused by it to life, agriculture, farm, and property. If we can quantify the rain-
fall most people can make some decisions before overwhelmed rain-aﬀected. The
contributions of this work are summarised as:
– We have assessed a pipeline of making choices for evaluating the ﬁnest rea-
sonable rain prediction.
– We have utilized 10 supervised regressors (Machine Learning Model). Be-
cause diﬀerent regressors give us diﬀerent results. So, it’s essential to ﬁnd
out the right model according to the requirements.<br>

In [44]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import ResponseMode

response_synthesizer = get_response_synthesizer(
    response_mode=ResponseMode.COMPACT
)

BM25_QUERY_ENGINE = RetrieverQueryEngine(
    retriever=bm25_retriever
)

In [46]:
response = BM25_QUERY_ENGINE.query("Why is rainfall forecasting important to prevent disasters?")
response

ResponseError: model requires more system memory (7.3 GiB) than is available (4.4 GiB)